In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import AUC
import pickle
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.image import load_img,img_to_array



In [2]:
def get_categories(data_dir):
    return [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

In [3]:
def prepare_data_generators(data_dir, image_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_generator = train_datagen.flow_from_directory(
        data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='training'
    )
    val_generator = train_datagen.flow_from_directory(
        data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='validation'
    )
    return train_generator, val_generator, train_datagen

In [4]:
def select_manual_test_images(data_dir, categories, num_test_images=5):
    manual_test_images = {}
    for category in categories:
        category_path = os.path.join(data_dir, category)
        images = [f for f in os.listdir(category_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        selected = random.sample(images, min(num_test_images, len(images)))
        manual_test_images[category] = selected
    with open("manual_test_images.pkl", "wb") as f:
        pickle.dump(manual_test_images, f)
    print("Manual test images saved:", manual_test_images)

In [5]:
def build_cnn_model(input_shape=(128, 128, 3), num_classes=10):
    model = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', AUC(name='auc')])
    return model

def f1_score_metric(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    return f1_score(y_true, y_pred, average='weighted')

In [6]:
def train_and_save_model(model, train_generator, val_generator, epochs=100, model_path="models/image_classifier/cnn_posture_classifier.h5", preprocessor_path="models/image_classifier/preprocessor.pkl"):
    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)
    model.save(model_path)
    with open(preprocessor_path, "wb") as f:
        pickle.dump(train_generator.image_data_generator, f)
    
    # Compute F1-score on validation data
    y_true = []
    y_pred = []
    for batch in val_generator:
        images, labels = batch
        preds = model.predict(images)
        y_true.extend(labels)
        y_pred.extend(preds)
        if len(y_true) >= val_generator.samples:
            break
    
    f1 = f1_score_metric(np.array(y_true), np.array(y_pred))
    print(f"Model and preprocessor saved! Validation F1-score: {f1:.4f}")

In [7]:
data_dir = "data/excercies_images"
image_size = (128, 128)
batch_size = 128
categories = get_categories(data_dir)
np.save("models/image_classifier/categories.npy",categories)
print(len(categories))
train_generator, val_generator, train_datagen = prepare_data_generators(data_dir, image_size, batch_size)
select_manual_test_images(data_dir, categories)
model = build_cnn_model(input_shape=(128, 128, 3), num_classes=len(categories))



22
Found 11090 images belonging to 22 classes.
Found 2763 images belonging to 22 classes.
Manual test images saved: {'barbell biceps curl': ['barbell biceps curl_4400191.jpg', 'barbell biceps curl_2100051.jpg', 'barbell biceps curl_5200301.jpg', 'barbell biceps curl_2400031.jpg', 'barbell biceps curl_5600041.jpg'], 'bench press': ['bench press_300051.jpg', 'bench press_3900451.jpg', 'bench press_300041.jpg', 'bench press_5100001.jpg', 'bench press_3200091.jpg'], 'chest fly machine': ['cfm_2600741.jpg', 'cfm_2700011.jpg', 'cfm_1500131.jpg', 'cfm_1500041.jpg', 'cfm_1200031.jpg'], 'deadlift': ['deadlift_1300201.jpg', 'deadlift_2500481.jpg', 'deadlift_1800001.jpg', 'deadlift_600001.jpg', 'deadlift_700021.jpg'], 'decline bench press': ['dbp_600481.jpg', 'dbp_300061.jpg', 'dbp_500401.jpg', 'dbp_200311.jpg', 'dbp_700251.jpg'], 'hammer curl': ['hammer curl_1700291.jpg', 'hammer curl_700441.jpg', 'hammer curl_1000041.jpg', 'hammer curl_1300071.jpg', 'hammer curl_400181.jpg'], 'hip thrust': ['hi

D:\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
print(type(train_generator))
train_and_save_model(model, train_generator, val_generator)


<class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>


D:\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 264s 3s/step - accuracy: 0.1549 - auc: 0.6457 - loss: 2.8706 - val_accuracy: 0.2827 - val_auc: 0.7945 - val_loss: 2.5337
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 248s 3s/step - accuracy: 0.7156 - auc: 0.9636 - loss: 1.0432 - val_accuracy: 0.4032 - val_auc: 0.8378 - val_loss: 2.4112
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - accuracy: 0.8794 - auc: 0.9895 - loss: 0.4583 - val_accuracy: 0.4430 - val_auc: 0.8304 - val_loss: 2.5904
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 280s 3s/step - accuracy: 0.9166 - auc: 0.9948 - loss: 0.3152 - val_accuracy: 0.4426 - val_auc: 0.8369 - val_loss: 2.8047
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 278s 3s/step - accuracy: 0.9351 - auc: 0.9972 - loss: 0.2234 - val_accuracy: 0.4770 - val_auc: 0.8396 - val_loss: 2.8027
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 278s 3s/step - accuracy: 0.9528 - auc: 0.9983 - loss: 0.1627 - val_accuracy: 0.4785 - val_auc: 0.8275 - val_loss: 3.0717
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1

KeyboardInterrupt: 

In [19]:
def predict_image(image_path, model_path="models/image_classifier/cnn_posture_classifier.h5", preprocessor_path="models/image_classifier/preprocessor.pkl", class_labels_path=None):
    model = keras.models.load_model(model_path)
    with open(preprocessor_path, "rb") as f:
        preprocessor = pickle.load(f)
    
    img = load_img(image_path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocessor.standardize(img_array)
    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    class_labels=np.load(class_labels_path)
    print(class_labels)
    print(predicted_class)
    class_name = class_labels[predicted_class] 
    
    print(f"Predicted class: {class_name}")
    return class_name


In [20]:
predict_image(image_path='data/excercies_images/barbell biceps curl//barbell biceps curl_4700091.jpg',class_labels_path="models/image_classifier/categories.npy")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
['barbell biceps curl' 'bench press' 'chest fly machine' 'deadlift'
 'decline bench press' 'hammer curl' 'hip thrust' 'incline bench press'
 'lat pulldown' 'lateral raises' 'leg extension' 'leg raises' 'plank'
 'pull up' 'push up' 'romanian deadlift' 'russian twist' 'shoulder press'
 'squat' 't bar row' 'tricep dips' 'tricep pushdown']
0
Predicted class: barbell biceps curl


np.str_('barbell biceps curl')